# Commons

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
from tqdm.notebook import tqdm

import os
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
!nvidia-smi

Tue Apr 27 18:23:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.14       Driver Version: 470.14       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   57C    P5    11W /  N/A |    816MiB /  8192MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
BASE_FOLDER = 'D:/Projects/CC_2/archive/'
IMAGES_FOLDER = BASE_FOLDER + 'images/'
LABELS_FOLDER = BASE_FOLDER + 'annotations/'

NO_CLASSES = 4
BATCH_SIZE = 8

In [4]:
def get_box(obj):
    x1 = int(obj.find('xmin').text)
    y1 = int(obj.find('ymin').text)
    x2 = int(obj.find('xmax').text)
    y2 = int(obj.find('ymax').text)
    return [x1, y1, x2, y2]

def get_label(obj):
    if obj.find('name').text == "with_mask":
        return 1
    elif obj.find('name').text == "mask_weared_incorrect":
        return 3
    return 2

def get_data(img_id, filename): 
    with open(filename) as fil:
        data = fil.read()
        soup = BeautifulSoup(data, 'xml')
        objs = soup.find_all('object')
        no_objs = len(objs)

        boxes = []
        labels = []
        for obj in objs:
            boxes.append(get_box(obj))
            labels.append(get_label(obj))
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        img_id = torch.tensor([img_id])

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = img_id
        
        return target

def collate_fn(batch):
    return tuple(zip(*batch))

def disp_img(img_tensor, result):
    fig,ax = plt.subplots(1)
    img = img_tensor.cpu().data

    ax.imshow(img.permute(1, 2, 0))
    no_boxes = len(result['boxes'])

    for i in range(no_boxes):
        if 'scores' in result:
            score = result['scores'][i].cpu().data
            if score < 0.2:
                continue
        
        box = result['boxes'][i].cpu().data
        x1, y1, x2, y2 = box

        label = result['labels'][i].cpu().data
        color = 'g'
        if label == 3:
            color = 'b'
        elif label  == 2:
            color = 'r'

        rect = patches.Rectangle((x1,y1),(x2-x1),(y2-y1),linewidth=1,edgecolor=color,facecolor='none')
        ax.add_patch(rect)

    plt.show()

In [5]:
class Dataset(object):
    def __init__(self, transforms):
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(IMAGES_FOLDER)))

    def __getitem__(self, idx):
        file_image = 'maksssksksss'+ str(idx) + '.png'
        file_label = 'maksssksksss'+ str(idx) + '.xml'
        img_path = os.path.join(IMAGES_FOLDER, file_image)
        label_path = os.path.join(LABELS_FOLDER, file_label)
        img = Image.open(img_path).convert("RGB")
        target = get_data(idx, label_path)
        
        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [6]:
def get_model(no_classes):
    model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, no_classes)

    return model

In [7]:
imgs = list(sorted(os.listdir(IMAGES_FOLDER)))
labels = list(sorted(os.listdir(LABELS_FOLDER)))

data_transform = transforms.Compose([transforms.ToTensor(),])

dataset = Dataset(data_transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

# Train

In [8]:
imgs = list(sorted(os.listdir(IMAGES_FOLDER)))
labels = list(sorted(os.listdir(LABELS_FOLDER)))

data_transform = transforms.Compose([transforms.ToTensor(),])

dataset = Dataset(data_transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [9]:
model = get_model(NO_CLASSES)
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(320,), max_size=640, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): ConvBNActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): Identity()
          )
        )
      )
      (2): InvertedResidual(
 

In [10]:
num_epochs = 200

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(data_loader)

for epoch in range(num_epochs):
    model.train()
    i = 0    
    epoch_loss = 0
    with tqdm(total=len(data_loader)) as pbar:
        for imgs, annotations in data_loader:
            i += 1
            imgs = list(img.to(device) for img in imgs)
            annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
            loss_dict = model(imgs, annotations)
            losses = sum(loss for loss in loss_dict.values())        

            optimizer.zero_grad()
            losses.backward()
            optimizer.step() 
            epoch_loss += losses
            pbar.update(1)

            del imgs
            del annotations
            torch.cuda.empty_cache()
    print(f'Epoch: {epoch}, Epoch Info: {epoch_loss}')
    torch.save(model.state_dict(), BASE_FOLDER + 'model.pt')

  0%|          | 0/107 [00:00<?, ?it/s]

Epoch: 0, Epoch Info: 75.91268157958984


  0%|          | 0/107 [00:00<?, ?it/s]

Epoch: 1, Epoch Info: 78.12786102294922


  0%|          | 0/107 [00:00<?, ?it/s]

...

Epoch: 197, Epoch Info: 46.48122024536133


  0%|          | 0/107 [00:00<?, ?it/s]

Epoch: 198, Epoch Info: 47.16146469116211


  0%|          | 0/107 [00:00<?, ?it/s]

Epoch: 199, Epoch Info: 49.84282684326172


In [13]:
model.eval()
for imgs, annotations in data_loader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        preds = model(imgs)

        for j in range(len(annotations)):
            print("Prediction")
            disp_img(imgs[j], preds[j])
            print("Target")
            disp_img(imgs[j], annotations[j])
            
        del imgs
        del annotations
        del preds
        torch.cuda.empty_cache()
        break

NameError: name 'model' is not defined

In [12]:
del model
torch.cuda.empty_cache()

# Loading

In [ ]:
!nvidia-smi

In [ ]:
imgs = list(sorted(os.listdir(IMAGES_FOLDER)))
labels = list(sorted(os.listdir(LABELS_FOLDER)))

data_transform = transforms.Compose([transforms.ToTensor(),])

dataset = Dataset(data_transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)

model2 = get_model(NO_CLASSES)
model2.load_state_dict(torch.load(BASE_FOLDER + 'model.pt'))
model2.eval()
model2.to(device)

In [ ]:
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    preds = model2(imgs)

    for j in range(len(annotations)):
        print("Prediction")
        disp_img(imgs[j], preds[j])
        print("Target")
        disp_img(imgs[j], annotations[j])

    del imgs
    del annotations
    del preds
    torch.cuda.empty_cache()
#     break